# Train a Model for Detecting Solar Panels

[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/geoai/blob/main/docs/examples/train_solar_panel_detection.ipynb)

## Install package
To use the `geoai-py` package, ensure it is installed in your environment. Uncomment the command below if needed.

In [ ]:
%pip install geoai-py

## Import libraries

In [ ]:
import geoai

## Download sample data

In [ ]:
train_raster_url = "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/solar_panels_davis_ca.tif"
train_vector_url = "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/solar_panels_davis_ca.geojson"
test_raster_url = "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/solar_panels_test_davis_ca.tif"

In [ ]:
train_raster_path = geoai.download_file(train_raster_url)
train_vector_path = geoai.download_file(train_vector_url)
test_raster_path = geoai.download_file(test_raster_url)

## Visualize sample data

In [8]:
geoai.print_raster_info(train_raster_path, show_preview=False)

===== RASTER INFORMATION: solar_panels_davis_ca.tif =====
Driver: GTiff
Dimensions: 6711 x 3242 pixels
Number of bands: 3
Data type: uint8
Coordinate Reference System: EPSG:3857
Georeferenced Bounds: BoundingBox(left=-13556075.706688922, bottom=4657729.5446374295, right=-13555574.76898035, top=4657971.533802349)
Pixel Resolution: 0.07464427187774597, 0.07464193859333511
NoData Value: None

----- Band Statistics -----
Band 1:
  Min: 0.00
  Max: 255.00
  Mean: 104.00
  Std Dev: 64.62
Band 2:
  Min: 0.00
  Max: 255.00
  Mean: 102.47
  Std Dev: 58.59
Band 3:
  Min: 0.00
  Max: 255.00
  Mean: 112.06
  Std Dev: 46.38


In [ ]:
geoai.view_raster(train_masks_url, )

In [ ]:
geoai.view_vector_interactive(train_vector_path, tiles=train_raster_url)

In [ ]:
geoai.view_raster(test_raster_url)

## Create training data

In [9]:
out_folder = "output"
tiles = geoai.export_geotiff_tiles(
    in_raster=train_raster_path,
    out_folder=out_folder,
    in_class_data=train_vector_path,
    tile_size=512,
    stride=256,
    buffer_radius=0,
)


Raster info for solar_panels_davis_ca.tif:
  CRS: EPSG:3857
  Dimensions: 6711 x 3242
  Resolution: (0.07464427187774597, 0.07464193859333511)
  Bands: 3
  Bounds: BoundingBox(left=-13556075.706688922, bottom=4657729.5446374295, right=-13555574.76898035, top=4657971.533802349)
Loaded 67 features from solar_panels_davis_ca.geojson
Vector CRS: EPSG:4326
Reprojecting features from EPSG:4326 to EPSG:3857


Generated: 312, With features: 137: 100%|██████████| 312/312 [00:16<00:00, 19.40it/s]


------- Export Summary -------
Total tiles exported: 312
Tiles with features: 137 (43.9%)
Average feature pixels per tile: 8422.7
Output saved to: output

------- Georeference Verification -------


## Train object detection model

In [ ]:
geoai.train_MaskRCNN_model(
    images_dir=f"{out_folder}/images",
    labels_dir=f"{out_folder}/labels",
    output_dir=f"{out_folder}/models",
    num_channels=3,
    pretrained=True,
    batch_size=4,
    num_epochs=100,
    learning_rate=0.005,
    val_split=0.2,
)

Using device: cuda
Found 312 image files and 312 label files
Training on 249 images, validating on 63 images
Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /root/.cache/torch/hub/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth


100%|██████████| 170M/170M [00:00<00:00, 182MB/s]


Epoch: 0, Batch: 0/63, Loss: 2.5700, Time: 4.52s
Epoch: 0, Batch: 10/63, Loss: 0.4559, Time: 9.40s
Epoch: 0, Batch: 20/63, Loss: 0.6814, Time: 9.64s
Epoch: 0, Batch: 30/63, Loss: 0.1426, Time: 9.72s
Epoch: 0, Batch: 40/63, Loss: 0.2866, Time: 9.78s
Epoch: 0, Batch: 50/63, Loss: 0.3084, Time: 9.77s
Epoch: 0, Batch: 60/63, Loss: 0.5514, Time: 9.68s
Epoch 1/100: Train Loss: 0.5558, Val Loss: inf, Val IoU: 0.2508
Saving best model with IoU: 0.2508
Epoch: 1, Batch: 0/63, Loss: 1.0403, Time: 2.98s
Epoch: 1, Batch: 10/63, Loss: 0.4080, Time: 10.15s
Epoch: 1, Batch: 20/63, Loss: 0.2563, Time: 10.22s
Epoch: 1, Batch: 30/63, Loss: 0.2540, Time: 10.35s
Epoch: 1, Batch: 40/63, Loss: 0.1587, Time: 10.12s
Epoch: 1, Batch: 50/63, Loss: 0.1623, Time: 10.01s
Epoch: 1, Batch: 60/63, Loss: 0.1966, Time: 10.22s
Epoch 2/100: Train Loss: 0.3388, Val Loss: inf, Val IoU: 0.3084
Saving best model with IoU: 0.3084
Epoch: 2, Batch: 0/63, Loss: 0.1435, Time: 1.91s
Epoch: 2, Batch: 10/63, Loss: 0.3745, Time: 10.31

## Run inference

In [ ]:
masks_path = "solar_panels_prediction.tif"
model_path = f"{out_folder}/models/best_model.pth"

In [ ]:
geoai.object_detection(
    test_raster_path,
    masks_path,
    model_path,
    window_size=512,
    overlap=256,
    confidence_threshold=0.5,
    batch_size=4,
    num_channels=3,
)

## Vectorize masks

In [ ]:
output_path = "solar_panels_prediction.geojson"
gdf = geoai.orthogonalize(masks_path, output_path, epsilon=2)

## Visualize results

In [ ]:
geoai.view_vector_interactive(output_path, tiles=test_raster_url)

In [ ]:
geoai.create_split_map(
    left_layer=output_path,
    right_layer=test_raster_url,
    left_args={"style": {"color": "red", "fillOpacity": 0.2}},
    basemap=test_raster_url,
)

![image](https://github.com/user-attachments/assets/69d20aec-991f-4681-b31e-fabcd7e21b91)